In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np

In [2]:
# Function to extract Product Title
def get_title(soup):

    try:
        # Outer Tag Object
        title = soup.find("span", attrs={"id":'productTitle'})
        
        # Inner NavigatableString Object
        title_value = title.text

        # Title as a string value
        title_string = title_value.strip()

    except AttributeError:
        title_string = ""

    return title_string

# Function to extract Product Price
def get_price(soup):

    try:
        price = soup.find("span", attrs={'id':'priceblock_ourprice'}).string.strip()

    except AttributeError:

        try:
            # If there is some deal price
            price = soup.find("span", attrs={'id':'priceblock_dealprice'}).string.strip()

        except:
            price = ""

    return price

# Function to extract Product Rating
def get_rating(soup):

    try:
        rating = soup.find("i", attrs={'class':'a-icon a-icon-star a-star-4-5'}).string.strip()
    
    except AttributeError:
        try:
            rating = soup.find("span", attrs={'class':'a-icon-alt'}).string.strip()
        except:
            rating = ""	

    return rating

# Function to extract Number of User Reviews
def get_review_count(soup):
    try:
        review_count = soup.find("span", attrs={'id':'acrCustomerReviewText'}).string.strip()

    except AttributeError:
        review_count = ""	

    return review_count

# Function to extract Availability Status
def get_availability(soup):
    try:
        available = soup.find("div", attrs={'id':'availability'})
        available = available.find("span").string.strip()

    except AttributeError:
        available = "Not Available"	

    return available

In [4]:
if __name__ == '__main__':

    # add your user agent 
    HEADERS = ({'User-Agent':'', 'Accept-Language': 'en-US, en;q=0.5'})

    # The webpage URL
    URL = "https://www.amazon.in/s?k=bags&crid=2M096C61O4MLT&qid=1653308124&sprefix=ba%2Caps%2C283&ref=sr_pg_1"

    # HTTP Request
    webpage = requests.get(URL, headers=HEADERS)

    # Soup Object containing all data
    soup = BeautifulSoup(webpage.content, "html.parser")

    # Fetch links as List of Tag Objects
    links = soup.find_all("a", attrs={'class':'a-link-normal s-no-outline'})

    # Store the links
    links_list = []

    # Loop for extracting links from Tag Objects
    for link in links:
            links_list.append(link.get('href'))

    d = {"title":[], "price":[], "rating":[], "reviews":[],"availability":[]}
    
    # Loop for extracting product details from each link 
    for link in links_list:
        new_webpage = requests.get("https://www.amazon.com" + link, headers=HEADERS)

        new_soup = BeautifulSoup(new_webpage.content, "html.parser")

        # Function calls to display all necessary product information
        d['title'].append(get_title(new_soup))
        d['price'].append(get_price(new_soup))
        d['rating'].append(get_rating(new_soup))
        d['reviews'].append(get_review_count(new_soup))
        d['availability'].append(get_availability(new_soup))

    
    amazon_df = pd.DataFrame.from_dict(d)
    amazon_df['title'].replace('', np.nan, inplace=True)
    amazon_df = amazon_df.dropna(subset=['title'])
    amazon_df.to_csv("amazon_data.csv", header=True, index=False)

In [5]:
amazon_df

,title,price,rating,reviews,availability
0,Red Lemon Unisex-adult Bange Series Rhombus Sh...,,4.4 out of 5 stars,67 ratings,Currently unavailable.
1,MOKOBARA Unisex-Adult Backpack Pink Vegan Leat...,,4.3 out of 5 stars,80 ratings,Currently unavailable.
2,Wesley Milestone 2.0 15.6 inch 30 L Casual Wat...,,4.3 out of 5 stars,"6,283 ratings",
4,"Skybags Brat Black 46 Cms Casual Backpack, Black",,4.1 out of 5 stars,"4,508 ratings",Currently unavailable.
5,TRUE HUMAN Emperor Anti-Theft backpack With US...,,3.8 out of 5 stars,"2,018 ratings",Currently unavailable.
7,Wesley Spartan Unisex Travel Hiking Laptop Bag...,,4.1 out of 5 stars,"1,271 ratings",Currently unavailable.
8,ADISA 15.6 inch Laptop Backpack Office Bag Col...,,3.9 out of 5 stars,790 ratings,Currently unavailable.
11,Fur Jaden Anti Theft Number Lock Backpack Bag ...,,3.9 out of 5 stars,"1,822 ratings",Currently unavailable.
14,SAFARI 15 Ltrs Sea Blue Casual Backpack (DAYPA...,,4.0 out of 5 stars,"18,660 ratings",Currently unavailable.
16,Fur Jaden Anti Theft Number Lock Backpack Bag ...,,3.9 out of 5 stars,"1,822 ratings",Currently unavailable.
